In [1]:
import os
from PIL import Image, ImageStat
import numpy as np
import requests
from io import BytesIO
import math

def enhance_image(image_path):
    # read image
    original_image = Image.open(image_path)

    # calculate brightness index
    brightness_index = ImageStat.Stat(original_image).mean[0]

    # check if image is bright enough
    if brightness_index >= 127:
        print("Skipping image", image_path, "because it is already bright enough")
        return original_image

    # determine amount of brightness enhancement needed
    enhancement_factor = (127 - brightness_index) / brightness_index
    print("Enhancing image", image_path, "by a factor of", enhancement_factor)

    # enhance image by the determined factor
    width, height = original_image.size
    pixels = original_image.load()
    for x in range(width):
        for y in range(height):
            rgb_tuple = pixels[x, y]
            new_r = min(255, int(rgb_tuple[0] * (1 + enhancement_factor)))
            new_g = min(255, int(rgb_tuple[1] * (1 + enhancement_factor)))
            new_b = min(255, int(rgb_tuple[2] * (1 + enhancement_factor)))
            pixels[x, y] = (new_r, new_g, new_b)
    enhanced_image = original_image

    return enhanced_image


def calculate_mse(original_image, enhanced_image):
    # calculate mean squared error (MSE)
    mse = np.mean((np.array(original_image) - np.array(enhanced_image)) ** 2)
    return mse

def calculate_psnr(original_image, enhanced_image):
    # calculate peak signal-to-noise ratio (PSNR) in decibels (dB)
    mse = calculate_mse(original_image, enhanced_image)
    if mse == 0:
        return float('inf')
    max_pixel = 255.0
    psnr = 20 * math.log10(max_pixel / math.sqrt(mse))
    return psnr

# input and output directories
input_dir = 'input'
output_dir = 'output'

# create output directory if it does not exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# initialize lists to store MSE and PSNR values for each image
mse_list = []
psnr_list = []
brightness_list = []

# get list of input filenames
input_filenames = [f for f in os.listdir(input_dir) if os.path.isfile(os.path.join(input_dir, f))]

# iterate over input images
for i, filename in enumerate(input_filenames):
    # enhance image
    enhanced_image = enhance_image(os.path.join(input_dir, filename))

    # calculate MSE and PSNR scores
    original_image = Image.open(os.path.join(input_dir, filename))
    mse = calculate_mse(original_image, enhanced_image)
    psnr = calculate_psnr(original_image, enhanced_image)
    brightness = ImageStat.Stat(original_image).mean[0]
    
    # append MSE and PSNR values to respective lists
    mse_list.append(mse)
    psnr_list.append(psnr)
    brightness_list.append(brightness)
    
    # save enhanced image
    output_path = os.path.join(output_dir, filename)
    if enhanced_image.mode == 'RGBA':
        enhanced_image = enhanced_image.convert('RGB')
    enhanced_image.save(output_path, format='JPEG')

    # print scores
    print("Image:", filename)
    print("MSE:", mse)
    print("PSNR:", psnr)
    print("Brightness index:", brightness)
    print("------------------------------------")

# calculate average MSE and PSNR values


Enhancing image input\0028506654_10.jpg by a factor of 15.725063021074755
Image: 0028506654_10.jpg
MSE: 118.08601467909568
PSNR: 27.408818951540507
Brightness index: 7.593394406943105
------------------------------------
Enhancing image input\2023-05-04_11-40-18.jpg by a factor of 6.276106536447548
Image: 2023-05-04_11-40-18.jpg
MSE: 43.57672450408528
PSNR: 31.738257779880314
Brightness index: 17.454389839377733
------------------------------------
Enhancing image input\2023-05-04_11-41-14.jpg by a factor of 17.332610531999197
Image: 2023-05-04_11-41-14.jpg
MSE: 99.56949136636636
PSNR: 28.14954072252696
Brightness index: 6.9275458494208495
------------------------------------
Enhancing image input\2023-05-04_11-41-38.jpg by a factor of 19.729084869590945
Image: 2023-05-04_11-41-38.jpg
MSE: 30.986990812795103
PSNR: 33.2190095748607
Brightness index: 6.126657341555192
------------------------------------
Enhancing image input\2023-05-04_11-42-01.jpg by a factor of 10.658744203516171
Imag